In [1]:
!nvidia-smi

Mon Dec  7 15:34:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    39W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import os

from pathlib import Path
base_path = str(Path.home())
print(base_path)

# Check if running in Colab or AWS.
#
import sys
IN_COLAB = 'google.colab' in sys.modules

/home/jupyter


In [4]:
import boto3

# List available buckets
#
available_buckets = []
s3 = boto3.resource('s3')
for idx, bucket in enumerate(s3.buckets.all()):
    print("Available bucket (%d): %s" % (idx, bucket.name))
    available_buckets.append(bucket.name)

# Interested in bucket with pre-trained models.
#
bucket_name = available_buckets[4]
print("\nDisplaying bucket: ", bucket_name)

!aws s3 ls s3://$bucket_name --recursive --human-readable --summarize

Available bucket (0): deep-learning-datasets-bucket
Available bucket (1): detectron2-models-bucket
Available bucket (2): gan-models-bucket
Available bucket (3): insightface-models-bucket
Available bucket (4): motion-models

Displaying bucket:  motion-models
2020-11-03 05:06:42    0 Bytes first-order-model/
2020-11-03 05:06:57    0 Bytes first-order-model/pytorch/
2020-11-17 15:28:56    0 Bytes first-order-model/pytorch/.ipynb_checkpoints/
2020-11-17 15:47:57    0 Bytes first-order-model/pytorch/log.txt
2020-11-17 15:47:55    0 Bytes first-order-model/pytorch/train-vis/
2020-11-17 15:47:42    1.7 KiB first-order-model/pytorch/vox-256-finetune.yaml
2020-11-03 05:10:01  716.1 MiB first-order-model/pytorch/vox-adv-cpk.pth.tar
2020-11-03 05:18:31  695.0 MiB first-order-model/pytorch/vox-cpk.pth.tar
2020-11-11 04:16:23    0 Bytes motion-cosegmentation/
2020-11-11 04:16:39    0 Bytes motion-cosegmentation/pytorch/
2020-11-11 04:18:57  716.3 MiB motion-cosegmentation/pytorch/taichi-10segments.

In [12]:
# Create a directory and mount bucket.
#
print("Mounting bucket: ", bucket_name)

local_path_to_motion_weights_dir = f'{base_path}/pretrained_models_motion' 
!mkdir -p $local_path_to_motion_weights_dir

model_to_mount = '/first-order-model/pytorch'
model_name = 'vox-cpk.pth.tar'
path_motion_weights = f'{local_path_to_motion_weights_dir}/{model_name}'

print("path to model: ", path_motion_weights)

Mounting bucket:  motion-models
path to model:  /home/jupyter/pretrained_models_motion/vox-cpk.pth.tar


In [10]:
import os

os.getuid()

1000

In [14]:
if IN_COLAB:
    !s3fs motion-models:/first-order-model/pytorch /content/pretrained_models_motion -o passwd_file=${HOME}/.passwd-s3fs -o umask=0277,uid=0
    print("Bucket mounted to: /content/pretrained_models_motion")
else:
    # AWS
    #
    # !s3fs gan-models-bucket:/pixel2style2pixel/pytorch /home/ubuntu/pretrained_models_p2p -o passwd_file=${HOME}/.passwd-s3fs -o umask=0277,uid=1000
    # print("Bucket mounted to: /home/ubuntu/pretrained_models_p2p")
    
    # GCP
    #
    !s3fs motion-models:/first-order-model/pytorch /home/jupyter/pretrained_models_motion -o passwd_file=${HOME}/.passwd-s3fs -o umask=0777,uid=1000
    print("Bucket mounted to: /home/jstaley/pretrained_models_motion")


Bucket mounted to: /home/jstaley/pretrained_models_motion_v2


## Load in the training data (i.e. movie clip) and save to PNGs.

In [7]:
def save(path, frames, format):
    if format == '.mp4':
        imageio.mimsave(path, frames)
    elif format == '.png':
        if os.path.exists(path):
            print ("Warning: skipping video %s" % os.path.basename(path))
            return
        else:
            os.makedirs(path)
        for j, frame in enumerate(frames):
            imageio.imsave(os.path.join(path, str(j).zfill(7) + '.png'), frames[j]) 
    else:
        print ("Unknown format %s" % format)
        exit()

In [8]:
import imageio
import numpy as np
from skimage.transform import resize
import matplotlib.pyplot as plt
import matplotlib.animation as animation

saving frames to:  /home/jstaley/assets/pngs_256px/train/id0002#11111111#000000#000000.mp4


saving frames to:  /home/jstaley/assets/pngs_256px/train/id0002#22222222#000000#000000.mp4


saving frames to:  /home/jstaley/assets/pngs_256px/train/id0002#33333333#000000#000000.mp4


## Finetune the checkpoint.

In [17]:
# Run in terminal
# NOTE:
# - Assumes config/vox-XXX.yaml has been appropriately updated/written
#   (e.g. update to epochs, data path, etc.)
#
# # Train from scratch (256).
# CUDA_VISILE_DEVICES=0 python run.py --mode train --config config/vox-256-finetune.yaml --checkpoint /home/jstaley/pretrained_models_motion_v2/vox-cpk.pth.tar --device_ids 0 --log_dir /home/jstaley/train_logs
#

# Train from checkpoint.
# i.e. run below in a terminal.
#
# CUDA_VISILE_DEVICES=0 python run.py --mode train --config config/vox-512.yaml --checkpoint /home/jupyter/pretrained_models/vox-cpk.pth.tar --device_ids 0 --log_dir /home/jupyter/train_logs